In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

In [ ]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = folder_path + '\\' + 'Inputs\\Inputs_TEST_GMI.xlsx'
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\100424-tests-conditions-GMI.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

componentSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Calibrage modèle multi-design

In [ ]:
# Permet de modifier une valeur du dictionnaire hyp
## Par exemple init_hyp('method_h_back_abs','free') va retourner le dictionnaire hyp avec la valeur 'method_h_back_abs' modifié en 'free'
def init_hyp(coeff, method) : 
    hyp[coeff] = method
    return(hyp)

Création de la matrice du modèle

In [ ]:
# Récupération des résultats expérimentaux

X_list_df = pd.read_excel(exp_path,header = 2) ## On ne prend pas en compe les 2 premières lignes // EN FONCTION DE COMMENT SONT PR2SENT2S LES RES
X_list = X_list_df.to_dict(orient='records')

In [ ]:
# Création de la matrice "A" sous forme de dictionnaire

measured_case = 0 # nb_measured_case = len(X_tot)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Dictionnaire des paramètres du modèle commun à tous les designs "alpha_0"
basic_parameters = dict([
    ('basic_parameters_1',1),
    ('basic_parameters_2',1),
    ('basic_parameters_3',1),
    ('basic_parameters_4',1),
    ('basic_parameters_5',1),
    ('basic_parameters_6',1),
    ('basic_parameters_7',1),
    ('basic_parameters_8',1),
    ('basic_parameters_9',1),
    ('basic_parameters_10',1),
    ])

# Matrice du modèle "A" 
model_matrix = dict(['alpha_0',basic_parameters])

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_tot = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = dict([
    ('tau_g_{measured_case}', X['componentSpecs']['pv']['tau_g']), # Transmittance glass
    ('alpha_g_{measured_case}', X['componentSpecs']['pv']['alpha_g']), # Absorptance glass
    ('eps_PV_{measured_case}', X['componentSpecs']['pv']['eps_PV']), # Émissivité cellule PV
    ('eps_g_{measured_case}', X['componentSpecs']['pv']['eps_g']), # Émissivité glass
    ('eps_hx_top_{measured_case}', X['componentSpecs']['main']['eps_hx_top']), # Émissivité de l'échangeur thermique sur le panneau
    ('eps_hx_back_{measured_case}', X['componentSpecs']['main']['eps_hx_back']), # Émissivité de l'échangeur thermique à l’arrière du panneau
    ('eps_ins_{measured_case}', X['componentSpecs']['main']['eps_ins']), # Émissivité isolant
    ('lambd_air_{measured_case}',  X['componentSpecs']['main']['lambd_air']) # Épaisseur de la couche d'air
    ])

model_matrix['alpha_{measured_case}'] = design_parameters
#fin boucle for

In [ ]:
# Création de la matrice "A" sous forme de listes

measured_case = 0 # nb_measured_case = len(X_list)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Matrice du modèle "A" 
model_matrix = np.ones(10)

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_list = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = np.zeros(10)
design_parameters[0] = X['componentSpecs']['pv']['tau_g'] # Transmittance glass
design_parameters[1] =  X['componentSpecs']['pv']['alpha_g'] # Absorptance glass
design_parameters[2] = X['componentSpecs']['pv']['eps_PV'] # Émissivité cellule PV
design_parameters[3] = X['componentSpecs']['pv']['eps_g'] # Émissivité glass
design_parameters[4] = X['componentSpecs']['main']['eps_hx_top'] # Émissivité de l'échangeur thermique sur le panneau
design_parameters[5] = X['componentSpecs']['main']['eps_hx_back'] # Émissivité de l'échangeur thermique à l’arrière du panneau
design_parameters[6] = X['componentSpecs']['main']['eps_ins'] # Émissivité isolant
design_parameters[7] = X['componentSpecs']['main']['lambd_air'] # Épaisseur de la couche d'air


model_matrix= np.vstack((model_matrix, design_parameters))

#fin boucle for
model_matrix

In [ ]:
# Création de la matrice de correspondance
correspondance_matrice = np.zeros((10,2))
# Deuxième écriture plus simple pour modifier les coefficients
correspondance_matrice = [
    ['coeff_h_back', 'eps_hx_back'],
    ['h_back0', 'eps_hx_top'],
    [0., 0.],
    [0., 0.],
    [0., 0.],
    [0., 0.],
    [0., 0.],
    [0., 0.],
    [0., 0.],
    [0., 0.]
    ]


In [ ]:
m = len(correspondance_matrice)
nb_measured_case = len(X_list)

# MAJ de hyp en fonction des méthodes que l'on veut
hyp = init_hyp("method_h_back_abs", "free")


def Qdot_model_tube_fluid(X_list, model_matrix, correspondance_matrice) :
        Qdot_list = []
        
        # Actualise le modèle
        for i in range(m) : 
                if correspondance_matrice[i, 0] != 0. : 
                       hyp[correspondance_matrice[i,0]] = model_matrix[i, 0]

        # Actualise la géométrie
        for j in range(nb_measured_case) : 
                for i in range(m) :
                        if correspondance_matrice[i, 0] != 0. :
                                X_list[j]['componentSpecs'][correspondance_matrice[i,1]] = model_matrix[i, j]
                df_one = ty.simu_one_steady_state_all_he(X_list[j]['componentSpecs'], X_list[j]['stepConditions'], hyp)[0]
                Qdot_list.append(df_one['Qdot_tube_fluid'])
        return(Qdot_list)